In [1]:
!pip install Arabic-Stopwords
!pip install qalsadi


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import requests
from bs4 import BeautifulSoup

import pyarabic.araby as araby
from nltk.corpus import stopwords # arabic stopwords
import arabicstopwords.arabicstopwords as stp # arabic stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import ArabicStemmer # Arabic Stemmer gets rot word
import qalsadi.lemmatizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
df = pd.read_csv("Arabic-Original.csv", header = None, names = ['text'])

In [4]:
df.head()

,text
0,1|1|بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
1,1|2|الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1|3|الرَّحْمَٰنِ الرَّحِيمِ
3,1|4|مَالِكِ يَوْمِ الدِّينِ
4,1|5|إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ


In [5]:
df = df['text'].str.split('|', expand=True)

In [6]:
df.columns = ['surah_num', 'ayah_num', 'ayah_txt']
df['surah_num'] = df['surah_num'].astype('int')
df['ayah_num'] = df['ayah_num'].astype('int')

In [7]:
df.head()

,surah_num,ayah_num,ayah_txt
0,1,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
1,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1,3,الرَّحْمَٰنِ الرَّحِيمِ
3,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   surah_num  6236 non-null   int32 
 1   ayah_num   6236 non-null   int32 
 2   ayah_txt   6236 non-null   object
dtypes: int32(2), object(1)
memory usage: 97.6+ KB


In [9]:
df.describe()

,surah_num,ayah_num
count,6236.000000,6236.000000
mean,33.519724,53.506575
std,26.461261,50.463924
min,1.000000,1.000000
25%,11.000000,16.000000
50%,26.000000,38.000000
75%,51.000000,75.000000
max,114.000000,286.000000


# Cleaning

In [10]:
def normalize_text(txt):
    txt = re.sub("[إأٱآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txr = re.sub("ة", "ه", txt)
    return txt

In [11]:
stopword = set(list(stp.stopwords_list())
              + stopwords.words('arabic'))
stopword = [normalize_text(word) for word in stopword]

In [12]:
#الهدف منه يرجع الكلمة لأصلها
st = ArabicStemmer()
lemmer = qalsadi.lemmatizer.Lemmatizer()

In [13]:
def clean_txt(txt):
    #إزالة التشكيل والتتويل
    txt = araby.strip_diacritics(txt)
    txt = araby.strip_tatweel(txt)
    
    
    txt = normalize_text(txt)
    
    #remove stop words and punctuation
    txt = ' '.join([token.translate(str.maketrans
    ('','',string.punctuation)) 
                    for token in txt.split(' ')
                   if token not in stopword])
    
    txt_lemmatized = ' '.join([lemmer.lemmatize(token) 
                 for token in txt.split(' ')])
    return txt+" "+txt_lemmatized
    

In [14]:
df['clean_txt'] = df['ayah_txt'].apply(lambda x: clean_txt(x))

In [15]:
df.sample(1)

,surah_num,ayah_num,ayah_txt,clean_txt
1155,7,202,وَإِخْوَانُهُمْ يَمُدُّونَهُمْ فِي الْغَيِّ ثُ...,واخوانهم يمدونهم الغي يقصرون واخوانهم أمد غي قصر


# Add Surah Names

In [16]:
surah_names = ['الفاتحة',
 'البقرة',
 'آل عمران',
 'النساء',
 'المائدة',
 'الأنعام',
 'الأعراف',
 'الأنفال',
 'التوبة',
 'يونس',
 'هود',
 'يوسف',
 'الرعد',
 'إبراهيم',
 'الحجر',
 'النحل',
 'الإسراء',
 'الكهف',
 'مريم',
 'طه',
 'الأنبياء',
 'الحج',
 'المؤمنون',
 'النور',
 'الفرقان',
 'الشعراء',
 'النمل',
 'القصص',
 'العنكبوت',
 'الروم',
 'لقمان',
 'السجدة',
 'الأحزاب',
 'سبأ',
 'فاطر',
 'يس',
 'الصافات',
 'ص',
 'الزمر',
 'غافر',
 'فصلت',
 'الشورى',
 'الزخرف',
 'الدخان',
 'الجاثية',
 'الأحقاف',
 'محمد',
 'الفتح',
 'الحجرات',
 'ق',
 'الذاريات',
 'الطور',
 'النجم',
 'القمر',
 'الرحمن',
 'الواقعة',
 'الحديد',
 'المجادلة',
 'الحشر',
 'الممتحنة',
 'الصف',
 'الجمعة',
 'المنافقون',
 'التغابن',
 'الطلاق',
 'التحريم',
 'الملك',
 'القلم',
 'الحاقة',
 'المعارج',
 'نوح',
 'الجن',
 'المزمل',
 'المدثر',
 'القيامة',
 'الإنسان',
 'المرسلات',
 'النبأ',
 'النازعات',
 'عبس',
 'التكوير',
 'الإنفطار',
 'المطففين',
 'الانشقاق',
 'البروج',
 'الطارق',
 'الأعلى',
 'الغاشية',
 'الفجر',
 'البلد',
 'الشمس',
 'الليل',
 'الضحى',
 'الشرح',
 'التين',
 'العلق',
 'القدر',
 'البينة',
 'الزلزلة',
 'العاديات',
 'القارعة',
 'التكاثر',
 'العصر',
 'الهمزة',
 'الفيل',
 'قريش',
 'الماعون',
 'الكوثر',
 'الكافرون',
 'النصر',
 'المسد',
 'الإخلاص',
 'الفلق',
 'الناس']

surah_names

['الفاتحة',
 'البقرة',
 'آل عمران',
 'النساء',
 'المائدة',
 'الأنعام',
 'الأعراف',
 'الأنفال',
 'التوبة',
 'يونس',
 'هود',
 'يوسف',
 'الرعد',
 'إبراهيم',
 'الحجر',
 'النحل',
 'الإسراء',
 'الكهف',
 'مريم',
 'طه',
 'الأنبياء',
 'الحج',
 'المؤمنون',
 'النور',
 'الفرقان',
 'الشعراء',
 'النمل',
 'القصص',
 'العنكبوت',
 'الروم',
 'لقمان',
 'السجدة',
 'الأحزاب',
 'سبأ',
 'فاطر',
 'يس',
 'الصافات',
 'ص',
 'الزمر',
 'غافر',
 'فصلت',
 'الشورى',
 'الزخرف',
 'الدخان',
 'الجاثية',
 'الأحقاف',
 'محمد',
 'الفتح',
 'الحجرات',
 'ق',
 'الذاريات',
 'الطور',
 'النجم',
 'القمر',
 'الرحمن',
 'الواقعة',
 'الحديد',
 'المجادلة',
 'الحشر',
 'الممتحنة',
 'الصف',
 'الجمعة',
 'المنافقون',
 'التغابن',
 'الطلاق',
 'التحريم',
 'الملك',
 'القلم',
 'الحاقة',
 'المعارج',
 'نوح',
 'الجن',
 'المزمل',
 'المدثر',
 'القيامة',
 'الإنسان',
 'المرسلات',
 'النبأ',
 'النازعات',
 'عبس',
 'التكوير',
 'الإنفطار',
 'المطففين',
 'الانشقاق',
 'البروج',
 'الطارق',
 'الأعلى',
 'الغاشية',
 'الفجر',
 'البلد',
 'الشمس',
 'الليل',
 'الضحى',
 

In [17]:
df['surah_names'] = df['surah_num'].apply(lambda x: surah_names[x-1])

In [18]:
df.sample(1)

,surah_num,ayah_num,ayah_txt,clean_txt,surah_names
3821,37,34,إِنَّا كَذَٰلِكَ نَفْعَلُ بِالْمُجْرِمِينَ,نفعل بالمجرمين فعل مجرم,الصافات


# Grouping by Surah name

In [19]:
group_surah = {'surah_num': [],'surah_name': [], 'surah_txt': []}

In [20]:
for i in range(1,df['surah_num'].max()+1):
    surah_txt = ' '.join(df[df['surah_num']==i]['ayah_txt'].to_list())
    group_surah['surah_txt'].append(surah_txt)
    group_surah['surah_num'].append(i)
    group_surah['surah_name'].append(surah_names[i-1])

In [21]:
df_surah = pd.DataFrame(group_surah)

In [22]:
df_surah.sample(1)

,surah_num,surah_name,surah_txt
102,103,العصر,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ وَالْعَ...


In [23]:
df_surah.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   surah_num   114 non-null    int64 
 1   surah_name  114 non-null    object
 2   surah_txt   114 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.8+ KB


# Similarity

In [24]:
# apply the TfidfVectorizer to the corpus
corpus = df['clean_txt']
vectorizer = TfidfVectorizer(ngram_range=(1,2))
corpus_vectorized = vectorizer.fit_transform(corpus)
print(corpus_vectorized.shape)

(6236, 75285)


In [25]:
vectorizer.get_feature_names_out()[:20]

array(['آتى', 'آتى أجزى', 'آتى اتوا', 'آتى اجرا', 'آتى اجرها',
       'آتى اجرهم', 'آتى اجوركم', 'آتى اجورهم', 'آتى احدا', 'آتى الاخرة',
       'آتى الله', 'آتى اوتي', 'آتى اوتيتم', 'آتى باذن', 'آتى تب',
       'آتى حذر', 'آتى حكم', 'آتى خير', 'آتى ردى', 'آتى زكاة'],
      dtype=object)

In [26]:
def show_best_results(df_quran, scores_array, threshold_low=0.881, threshold_high=0.990, top_n=3):
    # Reset the index of df_quran to ensure correct alignment
    df_quran_reset = df_quran.reset_index(drop=True)

    # Sort the indices based on scores_array
    sorted_indices = np.argsort(scores_array)[::-1]

    # Display only the top_n results within the specified score range
    displayed_count = 0
    for idx in sorted_indices:
        score = scores_array[idx]

        # Use iloc to access the DataFrame by integer position
        row = df_quran_reset.iloc[idx]
        ayah = row["ayah_txt"]
        ayah_num = row["ayah_num"]
        surah_name = row["surah_names"]

        if threshold_low <= score <= threshold_high:
            print(ayah)
            print(f'أيه رقم {ayah_num}  سورة {surah_name}')
            print(f"Similarity Score: {score:.4f}")
            print("====================================")

            displayed_count += 1
            if displayed_count >= top_n:
                break

In [27]:
def run_tfidf_optimized(name, threshold_low, threshold_high, top_n=5):
    # Filter rows based on the condition
    condition = df['surah_names'].str.contains(name)
    new_df = df[condition].copy()

    # Precompute TF-IDF vectors for the entire corpus
    corpus_vectorized = vectorizer.transform(df['clean_txt'])

    for _, row in new_df.iterrows():
        query = row['clean_txt']
        query_vectorized = vectorizer.transform([query])

        # Calculate cosine similarities using linear_kernel
        cosine_similarities = linear_kernel(query_vectorized, corpus_vectorized).flatten()

        # Pass the scores to the optimized show_best_results function
        show_best_results(df, cosine_similarities, threshold_low=threshold_low, threshold_high=threshold_high, top_n=top_n)

In [28]:
name=input(":اكتب اسم السوره المراده")
run_tfidf_optimized(name, threshold_low=0.555, threshold_high=1, top_n=5)

:اكتب اسم السوره المرادهالصافات
فَالزَّاجِرَاتِ زَجْرًا
أيه رقم 2  سورة الصافات
Similarity Score: 1.0000
فَالتَّالِيَاتِ ذِكْرًا
أيه رقم 3  سورة الصافات
Similarity Score: 1.0000
إِنَّ إِلَٰهَكُمْ لَوَاحِدٌ
أيه رقم 4  سورة الصافات
Similarity Score: 1.0000
رَبُّ السَّمَاوَاتِ وَالْأَرْضِ وَمَا بَيْنَهُمَا وَرَبُّ الْمَشَارِقِ
أيه رقم 5  سورة الصافات
Similarity Score: 1.0000
إِنَّا زَيَّنَّا السَّمَاءَ الدُّنْيَا بِزِينَةٍ الْكَوَاكِبِ
أيه رقم 6  سورة الصافات
Similarity Score: 1.0000
وَحِفْظًا مِنْ كُلِّ شَيْطَانٍ مَارِدٍ
أيه رقم 7  سورة الصافات
Similarity Score: 1.0000
دُحُورًا ۖ وَلَهُمْ عَذَابٌ وَاصِبٌ
أيه رقم 9  سورة الصافات
Similarity Score: 1.0000
إِلَّا مَنْ خَطِفَ الْخَطْفَةَ فَأَتْبَعَهُ شِهَابٌ ثَاقِبٌ
أيه رقم 10  سورة الصافات
Similarity Score: 1.0000
بَلْ عَجِبْتَ وَيَسْخَرُونَ
أيه رقم 12  سورة الصافات
Similarity Score: 1.0000
وَإِذَا ذُكِّرُوا لَا يَذْكُرُونَ
أيه رقم 13  سورة الصافات
Similarity Score: 1.0000
وَقَالُوا إِنْ هَٰذَا إِلَّا سِحْرٌ مُبِينٌ
أيه رقم 15  سورة الصافات


إِلَّا مَوْتَتَنَا الْأُولَىٰ وَمَا نَحْنُ بِمُعَذَّبِينَ
أيه رقم 59  سورة الصافات
Similarity Score: 1.0000
إِنَّ هَٰذَا لَهُوَ الْفَوْزُ الْعَظِيمُ
أيه رقم 60  سورة الصافات
Similarity Score: 1.0000
فَضْلًا مِنْ رَبِّكَ ۚ ذَٰلِكَ هُوَ الْفَوْزُ الْعَظِيمُ
أيه رقم 57  سورة الدخان
Similarity Score: 0.6031
أَذَٰلِكَ خَيْرٌ نُزُلًا أَمْ شَجَرَةُ الزَّقُّومِ
أيه رقم 62  سورة الصافات
Similarity Score: 1.0000
إِنَّا جَعَلْنَاهَا فِتْنَةً لِلظَّالِمِينَ
أيه رقم 63  سورة الصافات
Similarity Score: 1.0000
إِنَّهَا شَجَرَةٌ تَخْرُجُ فِي أَصْلِ الْجَحِيمِ
أيه رقم 64  سورة الصافات
Similarity Score: 1.0000
طَلْعُهَا كَأَنَّهُ رُءُوسُ الشَّيَاطِينِ
أيه رقم 65  سورة الصافات
Similarity Score: 1.0000
فَإِنَّهُمْ لَآكِلُونَ مِنْهَا فَمَالِئُونَ مِنْهَا الْبُطُونَ
أيه رقم 66  سورة الصافات
Similarity Score: 1.0000
فَمَالِئُونَ مِنْهَا الْبُطُونَ
أيه رقم 53  سورة الواقعة
Similarity Score: 0.6239
ثُمَّ إِنَّ مَرْجِعَهُمْ لَإِلَى الْجَحِيمِ
أيه رقم 68  سورة الصافات
Similarity Score: 1.0000
إِنَّهُمْ أَلْفَوْا 

سَلَامٌ عَلَىٰ إِبْرَاهِيمَ
أيه رقم 109  سورة الصافات
Similarity Score: 1.0000
إِنَّا كَذَٰلِكَ نَجْزِي الْمُحْسِنِينَ
أيه رقم 121  سورة الصافات
Similarity Score: 1.0000
إِنَّا كَذَٰلِكَ نَجْزِي الْمُحْسِنِينَ
أيه رقم 44  سورة المرسلات
Similarity Score: 1.0000
إِنَّا كَذَٰلِكَ نَجْزِي الْمُحْسِنِينَ
أيه رقم 131  سورة الصافات
Similarity Score: 1.0000
إِنَّا كَذَٰلِكَ نَجْزِي الْمُحْسِنِينَ
أيه رقم 80  سورة الصافات
Similarity Score: 1.0000
كَذَٰلِكَ نَجْزِي الْمُحْسِنِينَ
أيه رقم 110  سورة الصافات
Similarity Score: 1.0000
رَبِّ مُوسَىٰ وَهَارُونَ
أيه رقم 48  سورة الشعراء
Similarity Score: 1.0000
رَبِّ مُوسَىٰ وَهَارُونَ
أيه رقم 122  سورة الأعراف
Similarity Score: 1.0000
وَلَقَدْ مَنَنَّا عَلَىٰ مُوسَىٰ وَهَارُونَ
أيه رقم 114  سورة الصافات
Similarity Score: 1.0000
وَنَصَرْنَاهُمْ فَكَانُوا هُمُ الْغَالِبِينَ
أيه رقم 116  سورة الصافات
Similarity Score: 1.0000
وَهَدَيْنَاهُمَا الصِّرَاطَ الْمُسْتَقِيمَ
أيه رقم 118  سورة الصافات
Similarity Score: 1.0000
وَتَرَكْنَا عَلَيْهِ فِي الْآخِرِينَ
أ

أَمْ لَكُمْ سُلْطَانٌ مُبِينٌ
أيه رقم 156  سورة الصافات
Similarity Score: 1.0000
فَأْتُوا بِكِتَابِكُمْ إِنْ كُنْتُمْ صَادِقِينَ
أيه رقم 157  سورة الصافات
Similarity Score: 1.0000
وَجَعَلُوا بَيْنَهُ وَبَيْنَ الْجِنَّةِ نَسَبًا ۚ وَلَقَدْ عَلِمَتِ الْجِنَّةُ إِنَّهُمْ لَمُحْضَرُونَ
أيه رقم 158  سورة الصافات
Similarity Score: 1.0000
سُبْحَانَ اللَّهِ عَمَّا يَصِفُونَ
أيه رقم 159  سورة الصافات
Similarity Score: 1.0000
إِلَّا عِبَادَ اللَّهِ الْمُخْلَصِينَ
أيه رقم 160  سورة الصافات
Similarity Score: 1.0000
إِلَّا عِبَادَ اللَّهِ الْمُخْلَصِينَ
أيه رقم 74  سورة الصافات
Similarity Score: 1.0000
إِلَّا عِبَادَ اللَّهِ الْمُخْلَصِينَ
أيه رقم 40  سورة الصافات
Similarity Score: 1.0000
إِلَّا عِبَادَ اللَّهِ الْمُخْلَصِينَ
أيه رقم 128  سورة الصافات
Similarity Score: 1.0000
لَكُنَّا عِبَادَ اللَّهِ الْمُخْلَصِينَ
أيه رقم 169  سورة الصافات
Similarity Score: 0.7384
لَا أَعْبُدُ مَا تَعْبُدُونَ
أيه رقم 2  سورة الكافرون
Similarity Score: 0.6945
مَا أَنْتُمْ عَلَيْهِ بِفَاتِنِينَ
أيه رقم 162  سورة الص